In [1]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

def clean(text):
    return re.sub(r"\s+", " ", text.strip())

def synthesize(example):
    original = clean(example["translation"]["en"])
    degraded = re.sub(r"[.,?!:;\"'()]", "", original).lower()
    return {
        "input_text": degraded,
        "target_text": original
    }

def preprocess(example):
    prefix = "restore punctuation and capitalization: "
    input_text = prefix + example["input_text"]
    model_input = tokenizer(input_text, truncation=True, padding="max_length", max_length=64)
    label = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=64)
    model_input["labels"] = label["input_ids"]
    return model_input

def restore(text):
    prompt = "restore punctuation and capitalization: " + text.lower().strip()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        output = model.generate(**inputs, max_length=64)
    return tokenizer.decode(output[0], skip_special_tokens=True)

if __name__ == "__main__":
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
        print("Using device: MPS")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using device: CUDA")
    else:
        device = torch.device("cpu")
        print("Using device: CPU")

    # Load + clean + synthesize punctuation data
    raw_dataset = load_dataset("opus100", "en-fr", split="train[:5%]")

    filtered = [synthesize(x) for x in raw_dataset if x["translation"]["en"]]
    filtered = [x for x in filtered if x["input_text"] != x["target_text"]][:5000]
    dataset = Dataset.from_dict({
        "input_text": [x["input_text"] for x in filtered],
        "target_text": [x["target_text"] for x in filtered]
    })

    # Tokenize
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    model.to(device)

    tokenized = dataset.map(preprocess, batched=False, remove_columns=["input_text", "target_text"])
    split = tokenized.train_test_split(test_size=0.1)

    # Training config (10 epochs, use fp16 only on CUDA, else disabled)
    training_args = Seq2SeqTrainingArguments(
        output_dir="./t5_punct_model",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        logging_steps=50,
        save_total_limit=1,
        save_strategy="epoch",
        fp16=torch.cuda.is_available(),  
        predict_with_generate=True,
        report_to="none"
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=split["train"],
        eval_dataset=split["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )

    # Train and export model
    trainer.train()
    model.save_pretrained("t5_punct_model")
    tokenizer.save_pretrained("t5_punct_model")

    #  Test inference
    test_input = "its raining outside please take an umbrella with you and be careful"
    print("Input: ", test_input)
    print("Restored:", restore(test_input))


/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: MPS


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 5000/5000 [00:00<00:00, 7505.04 examples/s]
/var/folders/90/3mvx3ts92nsd32k5y3g748qc0000gn/T/ipykernel_8430/1065668042.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.


Step,Training Loss
50,2.806600
100,0.646700
150,0.454900
200,0.290900
250,0.256500
300,0.231100
350,0.200200
400,0.196000
450,0.206500
500,0.196100


/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/miniconda3/envs/final_project/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/minicon

Input:  its raining outside please take an umbrella with you and be careful
Restored: It's raining outside, please take an umbrella with you and be careful.


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model_dir = "t5_punct_model"

tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.eval()

# Example inference function (same as before)
def restore(text):
    prompt = "restore punctuation and capitalization: " + text.lower().strip()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        output = model.generate(**inputs, max_length=64)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test
print(restore("its raining outside please take an umbrella with you and be careful"))


It's raining outside, please take an umbrella with you and be careful.
